In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 198 kB 63.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=31add517ef261cdb420eda65dc8a58e1b15c63f2580fe0cbf6821e255e4f35ef
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [3]:
spark = SparkSession.builder.appName("LessPopularSuperhero").getOrCreate()

In [4]:
schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("name", StringType(), True)])

In [5]:
names = spark.read.schema(schema).option("sep", " ").csv("/content/drive/MyDrive/SparkCourse/Marvel-Names")

lines = spark.read.text("/content/drive/MyDrive/SparkCourse/Marvel-Graph")

In [6]:
# Small tweak vs. what's shown in the video: we trim each line of whitespace as that could
# throw off the counts.
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

In [9]:
lessPopular = connections.filter(func.col("connections") == 1).select("id","connections")
lessPopular.show()

+----+-----------+
|  id|connections|
+----+-----------+
|3967|          1|
|2180|          1|
|3447|          1|
|4819|          1|
|2532|          1|
|1877|          1|
|5286|          1|
|3793|          1|
|5189|          1|
|3518|          1|
|2914|          1|
|4364|          1|
| 620|          1|
|5246|          1|
|5425|          1|
|1200|          1|
|1167|          1|
|2072|          1|
|4184|          1|
|1890|          1|
+----+-----------+
only showing top 20 rows



In [44]:
lessPopularName = lessPopular.join(names, "id")

In [45]:
lessPopularName.select("id","name","connections").show()

+----+--------------------+-----------+
|  id|                name|connections|
+----+--------------------+-----------+
|3967|              NIMROD|          1|
|2180|                GOOM|          1|
|3447|         MARAUDER II|          1|
|4819|                SAJA|          1|
|2532|HOUSTON, LT. COMMAND|          1|
|1877|            FIREBOLT|          1|
|5286|SPARROW BEAR, MELLIS|          1|
|3793|               MOTH/|          1|
|5189|             SLAYER/|          1|
|3518| MASTER OF VENGEANCE|          1|
|2914|    KURLYCHEK, PATTY|          1|
|4364|   PROFESSOR GIBBON/|          1|
| 620|          BLOWTORCH/|          1|
|5246|    SNOW QUEEN/GITTE|          1|
|5425|       STERLING, SID|          1|
|1200|  COVEY, LAWRENCE K.|          1|
|1167|        COOPER, TERI|          1|
|2072|      GARNOK REBBAHN|          1|
|4184|        PEACEMONGER/|          1|
|1890|          FIREFLY II|          1|
+----+--------------------+-----------+
only showing top 20 rows



In [43]:
names.where(names.id == str(lessPopular.id).show()
#names.filter(func.col("id").isin(lessPopular["id"])).select("id","name").show()
#df.filter(col("state") === "OH").show(false)

SyntaxError: ignored